# Removing Watermark
> https://github.com/pymupdf/PyMuPDF/discussions/1855


In [ ]:
import sys
import fitz

word = "MY_SEARCH_STRING"  # search string
input_filname = "input.pdf"
output_filename = "output.pdf"
doc = fitz.open(input_filname)
assert isinstance(doc, fitz.Document)

In [61]:
for page in doc:
    page.clean_contents()
    xref = page.get_contents()[0]  # get xref of resulting /Contents object
    cont = bytearray(page.read_contents())  # read the contents source as a (modifiable) bytearray
    if cont.find(b"/Subtype/Watermark") > 0:  # this will confirm a marked-content watermark is present
        print(f"marked-content watermark present at {page.number}")
    while True:
        # start of definition. 
        i1 = cont.find(b"/Artifact <</Subtype")  # Need to incude <</Subtype because there are other /Artifacts that are not watermarks
        if i1 < 0: break  # none more left: done
        i2 = cont.find(b"EMC", i1)  # end of definition
        whitespace_len = i2 + 3 - (i1 - 2 + 1)
        # remove the full definition source "q ... EMC"
        # in the issue, the author merely removed with a blank string.
        # However, just removing the target string causes unexpected errors freezed operations
        # By replaciong the target string with the same length of whitespace, the program
        # avoids freezing and continued operations
        cont[i1 - 2:i2 + 3] = b" " * whitespace_len
        doc.update_stream(xref, cont)  # replace the original source

In [66]:
doc.save(output_filename, clean=True, garbage=3)  # save to new file
print("docsaved!")

[] b'q'
[0, 0.000183, 612, 792] b're'
[] b'W*'
[] b'n'
['/Artifact'] b'BMC'
[1] b'g'
[0] b'G'
[0, 0.000061, 612, 792] b're'
[] b'f*'
[] b'Q'
[] b'q'
[0.000009, 0, 612, 792] b're'
[] b'W*'
[] b'n'
[] b'EMC'
['/P', {'/MCID': 0}] b'BDC'
[] b'q'
[] b'BT'
[0] b'g'
[0] b'G'
[0] b'J'
[[], 0] b'd'
[0] b'j'
[1] b'w'
[10] b'M'
['/F1', 8.0399999999999991] b'Tf'
[0] b'Tc'
[0] b'Tw'
[1, 0, 0, 1, 0, 0] b'Tm'
[0] b'Tr'
['/RelativeColorimetric'] b'ri'
[1] b'i'
['/FXE3'] b'gs'
[383.35000000000002, 762.84000000000003] b'TD'
[['Windo', 9, 'w', 8, 's', 13, ' ', 49, 'Use', -6, 'r', -6, ' ', 49, 'Mo', -7, 'd', 4, 'e', -4, ' ', 49, 'Ex', -4, 'pl', 17, 'o', -6, 'i', 15, 't']] b'TJ'
Windo
[] b'ET'
[] b'Q'
[] b'EMC'
[] b'Q'
[] b'q'
[0.000009, 0, 612, 792] b're'
[] b'W*'
[] b'n'
['/P', {'/MCID': 1}] b'BDC'
[] b'q'
[] b'BT'
[0] b'g'
[0] b'G'
[0] b'J'
[[], 0] b'd'
[0] b'j'
[1] b'w'
[10] b'M'
['/F1', 8.0399999999999991] b'Tf'
[0] b'Tc'
[0] b'Tw'
[1, 0, 0, 1, 0, 0] b'Tm'
[0] b'Tr'
['/RelativeColorimetric'] b'ri'
[1]

TypeError: encoding without a string argument